In [1]:
from os import listdir
import pandas as pd
import numpy as np

filepaths = [f for f in listdir("./") if f.endswith('-clean.csv')]
li = []
for file in filepaths:
    df = pd.read_csv(file, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
cedulas = df.Cedula.unique();
excludeData = pd.read_csv("./2020-professions.csv", index_col=None)
exclude = excludeData['0926246299']
exclude = np.append(excludeData, [926246299])
indexArr = np.isin(cedulas, exclude)
cedulas = np.delete(cedulas, indexArr)
cedulas = cedulas[~np.isnan(cedulas)]


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
filepaths = [f for f in listdir("./") if f.endswith('-clean.csv')]
li = []
for file in filepaths:
    df = pd.read_csv(file, index_col=None, header=0,converters={'Cedula': lambda x: str(x)})
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)
df.to_csv("all_employees.csv", mode='w',index = False, header=False)

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.select import Select 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from time import sleep
from python_anticaptcha import AnticaptchaClient, ImageToTextTask
import os
from datetime import datetime
from PIL import Image
import requests

def solvingCaptcha(srcImage):
    
    api_key = 'ca211e61991cd07fe8601e05582b5eef'
    captcha_fp = open(srcImage, 'rb')
    client = AnticaptchaClient(api_key)
    task = ImageToTextTask(captcha_fp)
    job = client.createTask(task)
    job.join()
    
    return job.get_captcha_text()


def get_professions(driver, year):
    
    url = 'https://www.senescyt.gob.ec/consulta-titulos-web/faces/vista/consulta/consulta.xhtml'
    
    #I created a dataframe with few rucs. When you upload the csv you could add in the read_csv the argument "converters" in that way the ruc would keep showing the 0's
    #dfcontratos = pd.read_csv("C:/users/elcjo/hagp/contratos-unificado.csv",converters={'Ruc': lambda x: str(x)})
    #cedulas = ['0913637013']
    dftemprucs = pd.DataFrame(cedulas, columns=['Ruc'])
    
    driver.get(url)  # Abrir una nueva instancia del browser
    driver.maximize_window()
    x=0
     
    for index, row in dftemprucs.iterrows():
        #removing the alert displayed in the black box
        ruc = str(int(row["Ruc"]))
        if len(ruc) == 9 :
            ruc = "0"+ruc

        passport = driver.find_element_by_xpath("//input[@id='formPrincipal:identificacion']")
        passport.send_keys(ruc)
        
        #starting working to get the captcha
        file = open("Captcha.jpg", "wb")
        file.write(driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/main/form/div/div[2]/div/div/div[1]/table/tbody/tr[6]/td/div/div/div[1]/div[1]/img").screenshot_as_png)
        file.close()
        
        # Resolver captcha
        captcha = solvingCaptcha("Captcha.jpg")
        sleep(2)
        inputCaptcha = driver.find_element_by_xpath("//input[@id='formPrincipal:captchaSellerInput']")
        inputCaptcha.send_keys(captcha)

        #click on Buscar
        driver.find_element_by_xpath("//button[@id='formPrincipal:boton-buscar']").click()
                
        #if there is not results the titles will be fill with NA
        if len(driver.find_elements_by_xpath("//form[@id='formPrincipal']//div[3]")) != 0 :
            title1 = "NA"
            title2 = "NA"
            title_certificate="NA"

        # I did this to count how many titles have the employee
        count_of_divs = len(driver.find_elements_by_xpath("//span[@id='formPrincipal:groupDatos']/div"))
        count_of_divs_nontitle = len(driver.find_elements_by_xpath("//*[@id='formPrincipal:tablaStec']"))
        try:  #getting the titles
            
            # I am only going to work with two titles
            title1 = "NA"
            title2 = "NA"
            title_certificate = "NA"
            if count_of_divs_nontitle > 0:
                title_certificate = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/main/form/div/div[3]/div[2]/div/div/div/table/tbody/tr/td[2]').text
            if count_of_divs > 2:
                title1 = driver.find_element_by_xpath("//div[@class='contenido']//div[2]//div[2]//div[1]//div[1]//div[1]//table[1]//tbody[1]//tr[1]//td[1]").text
                title2 = driver.find_element_by_xpath("//div[3]//div[2]//div[1]//div[1]//div[1]//table[1]//tbody[1]//tr[1]//td[1]").text
            # this is when the employee has just one title
            if count_of_divs == 2:
                title1 = driver.find_element_by_xpath("//div[@class='contenido']//div[2]//div[2]//div[1]//div[1]//div[1]//table[1]//tbody[1]//tr[1]//td[1]").text   
                title2 = "NA"
            
        except:
            title1 = "NA"
            title2 = "NA"
            title_certificate="NA"

        #creating a serie
        values_to_add = {'Ruc':[str(ruc).strip()], 'Title1':[title1],'Title2':[title2],'Certificate': [title_certificate]}
        row_to_add = pd.DataFrame(values_to_add, dtype = str)
        row_to_add.to_csv(year + "-professions.csv",mode='a',index = False, header=False)
        #df = df.append(row_to_add) #adding the data to the dataframe
          
    #you could change the path and save the dataframe in a csv file    
    #df.to_csv( year + "-professions.csv")
    my_driver.close()
 

options = Options()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
my_driver = webdriver.Chrome(options=options,executable_path=r'./chromedriver')
 
get_professions(my_driver, '2020')


In [13]:
import pandas as pd
professions = pd.read_csv("./2020-professions.csv", converters={'Cedula': lambda x: str(x)}, index_col=None, names=['Cedula', 'Titulo 1', 'Titulo 2', 'Titulo 3'])
#Merge employess and res
employeesProfessions = df.merge(professions, how="left", left_on='Cedula', right_on='Cedula')
employeesProfessions.to_csv("./employees-professions.csv",mode='w',index = False, header=False )
degree_columns = employeesProfessions[employeesProfessions.columns[len(employeesProfessions.columns)-3:]];
employeesProfessions.insert(len(employeesProfessions.columns),'hasDegree',0)
for index, row in degree_columns.iterrows():
    employeesProfessions.loc[index, 'hasDegree'] =  1 if row.any() else 0


In [18]:
df = employeesProfessions[employeesProfessions['hasDegree'] == 0].groupby(['Puesto 1','Cedula']).agg({'hasDegree':['sum','count']})
df


hasDegree      
                                                                   sum count
Puesto 1                                           Cedula                   
ALISTA DE PLANIFICACION, SEGUIMIENTO Y EVALUACI... 926430596         0     1
ANALISTA  ASESORIA JURIDICA 1                      201995917         0     1
                                                   913726378         0     4
                                                   917858201         0     1
                                                   925162174         0     1
...                                                                ...   ...
UNIDADES DE ESPECIALIDADES CLINICAS Y/O QUIRURG... 703352799         0     1
                                                   920954641         0     1
VASQUEZ MORA JAVIER VICENTE                                          0     1
VILLAO MORAN JOSE GABRIEL                                            0     1
ZAMBRANO SANCHEZ JOSUE EDUARDO                                       0     1

[2705 rows x 2 columns]